In [1]:
import os, sys, csv, json, re
import parseidf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Write Baseline IDF

In [3]:
WORK_FOLDER_NAME = 'SF_100'
BLDG_NAME_LIST = os.listdir(WORK_FOLDER_NAME)

for bldg_name in BLDG_NAME_LIST:
    input_file_path = os.path.join(WORK_FOLDER_NAME, bldg_name, 'model.idf')
    output_file_path= os.path.join(WORK_FOLDER_NAME, bldg_name, 'model-1.idf')
    
    if not os.path.exists(input_file_path):
        continue

    with open(input_file_path, 'r') as f:
        idf = parseidf.parse(f.read())
        ext_surfaces = []
        
        # Change run period
        idf['RunPeriod'.upper()][0][2] = 9
        idf['RunPeriod'.upper()][0][3] = 1
        idf['RunPeriod'.upper()][0][5] = 9
        idf['RunPeriod'.upper()][0][6] = 2
        
        # Get list of all ext surfaces
        for surface_detailed in idf['BuildingSurface:Detailed'.upper()]:
            if surface_detailed[5] == 'Outdoors':
                ext_surfaces.append(surface_detailed[1])
        for surface_detailed in idf['FenestrationSurface:Detailed'.upper()]:
            ext_surfaces.append(surface_detailed[1])

        # Add meter and reportable variables
        idf['Output:Meter'.upper()] = list()
        idf['Output:Meter'.upper()].append(['Output:Meter', 'Cooling:Electricity', 'hourly'])
        idf['Output:Meter'.upper()].append(['Output:Meter', 'Heating:Gas', 'hourly'])
        idf['Output:Meter'.upper()].append(['Output:Meter', 'Electricity:Facility', 'hourly'])
        idf['Output:Meter'.upper()].append(['Output:Meter', 'Gas:Facility', 'hourly'])
        idf['Output:Variable'.upper()] = list()
        for surf in sorted(ext_surfaces):
            idf['Output:Variable'.upper()].append(
                ['Output:Variable', surf, 'Surface Outside Face Temperature', 'hourly'])
            idf['Output:Variable'.upper()].append(
                ['Output:Variable', surf, 'Surface Outside Face Convection Heat Transfer Coefficient', 'hourly'])
            idf['Output:Variable'.upper()].append(
                ['Output:Variable', surf, 'Surface Outside Face Heat Emission to Air Rate', 'hourly'])
        
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Total Zone Exfiltration Heat Loss', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'HVAC System Total Heat Rejection Energy', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Air System Relief Air Total Heat Loss Energy', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Outdoor Air Drybulb Temperature', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Outdoor Air Dewpoint Temperature', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Outdoor Air Wetbulb Temperature', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Outdoor Air Relative Humidity', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Outdoor Air Barometric Pressure', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Wind Speed', 'hourly'])
        idf['Output:Variable'.upper()].append(
            ['Output:Variable', '*', 'Site Wind Direction', 'hourly'])
        
        # to CSV output directly
        idf['OutputControl:Files'.upper()] = list()
        idf['OutputControl:Files'.upper()].append(['OutputControl:Files', 'Yes', 'No', 'No', 'No'])


    with open(output_file_path, 'w') as out:
        for idf_object_key in idf:
            idf_objects = idf[idf_object_key]
            for idf_object in idf_objects:
                for attr_num in range(len(idf_object)):
                    if attr_num == 0:
                        out.write('  ' + str(idf_object[attr_num]) + ',\n')
                    elif attr_num == len(idf_object) - 1:
                        out.write('    ' + str(idf_object[attr_num]) + ';\n')
                    else:
                        out.write('    ' + str(idf_object[attr_num]) + ',\n')
                out.write('\n')